# Sequential feature selection algorithms


An alternative way to reduce complexity of the model to avoid overfitting is dimensionality reduction via feature selection, very useul for unregularized models. <br>
There is 2 way, feature selection and extraction. <br>
Feature selection -> select a subset of the original features. <br>
Feature extraction -> we derive information from the feature set to construct a new feature space. <br>


<H1> Greedy search algorithms <br>
<h6> Sequential feature selection is a family of greedy search algorithms. <br>
Having a d-dimension feature space to k-dimension where k < d. <br>
Select the subset of features that are most relevant to the problem.
<br> A classical algorithm is sequential backward selection(SBS). This reduce the dimension of the initial feature subspace with a minimum decay in the performance. N.B. greedy search only for local optimum. <br>
We need define criterion function J to minimize.
at each stage we eliminate the feature that causes the least performance loss after removal. four simple steps: <br>

1.   Initialize algorithm with k = d.
1.   Determine feature that maximize the criterion
2.   Remove the feature that maximize the criterion.
2.   If k = nDesiredFeatures end, otherwise go to 2.



In [2]:
from sklearn.base import clone
from itertools import combinations
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
class SBS:
  def __init__(self, estimator, k_features,
                scoring=accuracy_score,
                test_size=0.25, random_state=1):
    self.scoring = scoring
    self.estimator = clone(estimator)
    self.k_features = k_features
    self.test_size = test_size
    self.random_state = random_state

  def fit(self, X, y):
    X_train, X_test, y_train, y_test = \
      train_test_split(X, y, test_size=self.test_size,
      random_state=self.random_state)
    dim = X_train.shape[1]
    self.indices_ = tuple(range(dim))
    self.subsets_ = [self.indices_]
    score = self._calc_score(X_train, y_train,
                              X_test, y_test, self.indices_)
    self.scores_ = [score]
    while dim > self.k_features:
      scores = []
      subsets = []
      for p in combinations(self.indices_, r=dim - 1):
        score = self._calc_score(X_train, y_train,
                                 X_test, y_test, p)
        scores.append(score)
        subsets.append(p)
      best = np.argmax(scores)
      self.indices_ = subsets[best]
      self.subsets_.append(self.indices_)
      dim -= 1
      self.scores_.append(scores[best])
    self.k_score_ = self.scores_[-1]
    return self

  def transform(self, X):
    return X[:, self.indices_]
  def _calc_score(self, X_train, y_train, X_test, y_test, indices):
    self.estimator.fit(X_train[:, indices], y_train)
    y_pred = self.estimator.predict(X_test[:, indices])
    score = self.scoring(y_test, y_pred)
    return score
